In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
import nipype.interfaces.mrtrix3 as mtx
import nipype.interfaces.mrtrix.convert as mtxc
import nipype.interfaces.fsl as fsl
from nipype.workflows.dmri.mrtrix.diffusion import create_mrtrix_dti_pipeline
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd, makedirs
from shutil import copyfile
from nipype import config, logging
from datetime import date
import AFQ
today = str(date.today())
config.enable_debug_mode()

/usr/local/anaconda3/lib/python3.6/site-packages/nipype/workflows/dmri/mrtrix/group_connectivity.py:16: UserWarning: cmp not installed
  warnings.warn('cmp not installed')


In [2]:
#Set user and path variables

user = expanduser('~')
if user == '/Users/lucindasisk':
    home = join(user, 'Desktop/Milgram/candlab')
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi')
    workflow_dir = join(home, 'analyses/shapes/dwi/tractography_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/tractography_data')
else:
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi/eddyCUDA_data')
    workflow_dir = join(home, 'analyses/shapes/dwi/tractography_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/tractography_data')
    
# Read in subject subject_list
# subject_info = read_csv(
#     home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ', header=None)
# subject_list = subject_info[0].tolist()

# Manual subject list
subject_list = ['sub-A200', 'sub-A201']

In [3]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir,
                        substitutions = [('_subject_id_', '')]),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(dti = join(proc_dir,'eddyCUDA_data/3_Eddy_Corrected/{subject_id}/eddy_corrected.nii.gz'),
                bval = join(raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec = join(proc_dir,'eddyCUDA_data/3_Eddy_Corrected/{subject_id}/eddy_corrected.eddy_rotated_bvecs'),
                t1 = join(proc_dir, 'preproc_data/2_Transfer/{subject_id}/{subject_id}_ses-shapesV1_T1w_resample_brain.nii.gz'),
                aseg = join(home, 'data/mri/hcp_pipeline_preproc/shapes/{subject_id}/MNINonLinear/aparc.a2009s+aseg.nii.gz')
               )

sf = Node(SelectFiles(template, 
                      base_directory = home),
          name = 'sf')
                

### Pull aseg file and extract ROIs

In [ ]:
# Reorient topup b0 image to std
reorient1 = Node(fsl.Reorient2Std(output_type='NIFTI_GZ'),
                 name='reorient1')

#Register aseg to T1
register1 = Node(fsl.FLIRT(out_matrix_file='asegtoT1_reorient_reg.mat',
                           rigid2D=True,
                           output_type='NIFTI_GZ'),
                 name='register1')

#Threshold left hippocampus and amygdala (hippocampus = 17, amygdala = 18)
thresh1 = Node(fsl.Threshold(thresh = 17,
                           direction = 'below'),
              name='thresh1')
thresh2 = Node(fsl.Threshold(thresh = 19,
                             direction = 'above',
                             out_file='Left_Hipp_Amyg_ROI.nii.gz'),
              name='thresh2')

#Threshold right hippocampus and amygdala (hippocampus =53, amygdala = 54)
thresh3 = Node(fsl.Threshold(thresh = 52,
                           direction = 'below'),
              name='thresh3')
thresh4 = Node(fsl.Threshold(thresh = 54,
                             direction = 'above',
                             out_file='Right_Hipp_Amyg_ROI.nii.gz'),
              name='thresh4')

#Threshold left OFC (Lh G Orbital Cortex = 11124)
thresh5 = Node(fsl.Threshold(thresh = 11123,
                             direction = 'below'),
              name='thresh5')
thresh6 = Node(fsl.Threshold(thresh = 11124,
                             direction = 'above',
                             out_file='Left_OFC_ROI.nii.gz'),
              name='thresh6')

#Threshold right OFC (Rh G Orbital Cortex = 11124)
thresh7 = Node(fsl.Threshold(thresh = 12123,
                             direction = 'below'),
              name='thresh7')
thresh8 = Node(fsl.Threshold(thresh = 12124,
                             direction = 'above',
                             out_file='Right_OFC_ROI.nii.gz'),
              name='thresh8')

In [ ]:
roi_flow = Workflow(name = 'roi_flow')
roi_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                  (sf, reorient1, [('aseg', 'in_file')]),
                  (reorient1, register1, [('out_file', 'in_file')]),
                  (sf, register1, [('t1', 'reference')]),
                  (register1, datasink, [('out_file', '4_ROIs')]),
                  (register1, thresh1, [('out_file', 'in_file')]),
                  (thresh1, thresh2, [('out_file', 'in_file')]),
                  (register1, thresh3, [('out_file', 'in_file')]),
                  (thresh3, thresh4, [('out_file', 'in_file')]),
                  (thresh2, datasink, [('out_file', '4_ROIs.@par')]),
                  (thresh4, datasink, [('out_file', '4_ROIs.@par.@par')]),
                  (register1, thresh5, [('out_file', 'in_file')]),
                  (thresh5, thresh6, [('out_file', 'in_file')]),
                  (thresh6, datasink, [('out_file', '4_ROIs.@par.@par.@par')]),
                  (register1, thresh7, [('out_file', 'in_file')]),
                  (thresh7, thresh8, [('out_file', 'in_file')]),
                  (thresh8, datasink, [('out_file', '4_ROIs.@par.@par.@par.@par')])
                 ])
                  
roi_flow.base_dir = workflow_dir
roi_flow.write_graph(graph2use = 'flat')
dwi = roi_flow.run('MultiProc', plugin_args={'n_procs': 4})

### Nodes for Diffusion workflow

In [4]:
#Generate binary mask
bet=Node(fsl.BET(frac=0.2,
                mask=True),
        name='bet')

#Generate 5 tissue type (5tt) segmentation using FAST algorithm
seg5tt = Node(mtx.Generate5tt(algorithm = 'fsl',
                             out_file = 'T1s_5tt_segmented.nii.gz'),
             name='seg5tt')

#Estimate response functions for spherical deconvolution using the specified algorithm (Dhollander)
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/preprocess.html#responsesd
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/response_function_estimation.html#response-function-estimation
#Max_sh (lmax variable) determined in shell order from here: https://mrtrix.readthedocs.io/en/3.0_rc2/constrained_spherical_deconvolution/lmax.html
#DWI has 5 shells: 7 b0 volumes, 6 b500 vols, 15 b1000 vols, 15 b2000 bols, 60 b3000 vols
dwiresp = Node(mtx.ResponseSD(algorithm = 'dhollander',
                              max_sh=[0,2,4,4,8],
                              wm_file = 'wm_response.txt',
                              gm_file = 'gm_response.txt',
                              csf_file = 'csf_response.txt'),
              name='dwiresp')

#Estimate fiber orientation distributions from diffusion data sing spherical deconvolution
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/reconst.html
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/multi_shell_multi_tissue_csd.html
#Max SH here determined by tissue type - chose 8,8,8 per forum recommendations
mscsd = Node(mtx.EstimateFOD(algorithm = 'msmt_csd',
                             bval_scale = 'yes',
                            max_sh = [8,8,8]),
            name='mscsd')

#Perform Tractography - ACT using iFOD2 (https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/tracking.html) 
tract = Node(mtx.Tractography(algorithm='iFOD2',
                             n_trials=10000,
                             out_file='whole_brain_trcktography.tck'),
            name='tract')

trkconvert = Node(mtxc.MRTrix2TrackVis(out_filename = 'whole_brain_tractography_converted.trk'),
                 name='trkconvert')

In [ ]:
tract_flow = Workflow(name = 'tract_flow')
tract_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                    (sf, bet, [('t1', 'in_file')]),
                    (sf, seg5tt, [('t1', 'in_file')]),
                    (seg5tt, datasink, [('out_file', '4_tract_Reconstruction')]),
                    (sf, dwiresp, [('dti', 'in_file'),
                                   ('bval','in_bval'),
                                   ('bvec', 'in_bvec')]),
#                     (dwiresp, datasink, [('wm_file', '4_tract_Reconstruction.@par'),
#                                         ('gm_file', '4_tract_Reconstruction.@par.@par'),
#                                         ('csf_file', '4_tract_Reconstruction.@par.@par.@par')]),
                    (sf, mscsd, [('dti', 'in_file'),
                                ('bval', 'in_bval'),
                                ('bvec', 'in_bvec')]),
                    (dwiresp, mscsd, [('wm_file', 'wm_txt'),
                                      ('gm_file', 'gm_txt'),
                                      ('csf_file', 'csf_txt')]),
                    (seg5tt, tract, [('out_file', 'act_file')]),
                    (mscsd, tract, [('wm_odf', 'in_file')]),
#                     (mscsd, datasink, [('wm_odf', '4_tract_Reconstruction.@par.@par.@par.@par'),
#                                        ('gm_odf', '4_tract_Reconstruction.@par.@par.@par.@par.@par'),
#                                        ('csf_odf','4_tract_Reconstruction.@par.@par.@par.@par.@par.@par')]),
                    (sf, tract, [('bval', 'in_bval'),
                                 ('bvec', 'in_bvec')]),
                    (bet, tract, [('mask_file', 'seed_image')]),
                    (tract, trkconvert, [('out_file', 'in_file')]),
                    (sf, trkconvert, [('t1','image_file')]),
                    (sf, trkconvert, [('t1','registration_image_file')]),
                    (trkconvert, datasink, [('out_file', '4_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par')]),
                    (tract, datasink, [('out_file', '4_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par.@par')]),      
                    (bet, datasink, [('mask_file','4_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par.@par.@par')])
                   ])
tract_flow.base_dir = workflow_dir
tract_flow.write_graph(graph2use = 'flat')
dwi = tract_flow.run('MultiProc', plugin_args={'n_procs': 4})

190920-09:26:32,111 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): No edge data
190920-09:26:32,113 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190920-09:26:32,114 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): No edge data
190920-09:26:32,115 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): new edge data: {'connect': [('t1', 'in_file')]}
190920-09:26:32,115 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): No edge data
190920-09:26:32,116 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): new edge data: {'connect': [('t1', 'in_file')]}
190920-09:26:32,117 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): No edge data
190920-09:26:32,118 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): new edge data: {'connect': [('out_file', '4_tract_Reconstruction')]}
190920-09:26:32,118 nipype.workflow DEBUG:
	 (tract_flow.sf, t

190920-09:26:32,742 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A200
190920-09:26:32,742 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
190920-09:26:32,889 nipype.workflow DEBUG:
	 Performing depth first search
190920-09:26:32,943 nipype.workflow INFO:
	 Running in parallel.
190920-09:26:32,947 nipype.workflow DEBUG:
	 Progress: 16 jobs, 0/0/2 (done/running/ready), 0/16 (pending_tasks/waiting).
190920-09:26:32,948 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
190920-09:26:32,949 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 4/4.
190920-09:26:33,19 nipype.workflow DEBUG:
	 Allocating tract_flow.sf ID=0 (0.20GB, 1 threads). Free: 28.60GB, 3 threads.
190920-09:26:33,20 nipype.workflow DEBUG:
	 Setting node inputs
190920-09:26:33,44 nipype.workflow DEBUG:
	 [Node] Hashes: [('base_directory', '/Users/lucindasisk/Desktop/Milgram/candlab

	 Needed files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bval;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.eddy_rotated_bvecs;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_data/2_Transfer/sub-A200/sub-A200_ses-shapesV1_T1w_resample_brain.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/hcp_pipeline_preproc/shapes/sub-A200/MNINonLinear/aparc.a2009s+aseg.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/sf/_0xb55969a6e4f3187f1953032d2972e1b0_unfinished.json;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/sf/_inputs.pklz;/Users/lucindasisk/

190920-09:26:35,157 nipype.workflow DEBUG:
	 [Node] Up-to-date cache found for "tract_flow.seg5tt".
190920-09:26:35,158 nipype.workflow DEBUG:
	 Checking hash "tract_flow.seg5tt" locally: cached=True, updated=True.
190920-09:26:35,158 nipype.workflow DEBUG:
	 Skipping cached node tract_flow.seg5tt with ID 3.
190920-09:26:35,159 nipype.workflow INFO:
	 [Job 3] Cached (tract_flow.seg5tt).
190920-09:26:35,160 nipype.workflow DEBUG:
	 Allocating tract_flow.bet ID=4 (0.20GB, 1 threads). Free: 28.20GB, 1 threads.
190920-09:26:35,161 nipype.workflow DEBUG:
	 Setting node inputs
190920-09:26:35,162 nipype.workflow DEBUG:
	 input: in_file
190920-09:26:35,163 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/result_sf.pklz
190920-09:26:35,163 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A

190920-09:26:37,31 nipype.workflow DEBUG:
	 input: in_bvec
190920-09:26:37,31 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/result_sf.pklz
190920-09:26:37,31 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/result_sf.pklz
190920-09:26:37,37 nipype.workflow DEBUG:
	 output: bvec
190920-09:26:37,37 nipype.workflow DEBUG:
	 [Node] mscsd - setting input in_bvec = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A201/eddy_corrected.eddy_rotated_bvecs
190920-09:26:37,38 nipype.workflow DEBUG:
	 input: wm_txt
190920-09:26:37,39 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/dwiresp/result_dwiresp.pklz
190920-09:26:37,39 ni

	 [Node] Hashes: [('algorithm', 'msmt_csd'), ('bval_scale', 'yes'), ('csf_odf', 'csf.mif'), ('csf_txt', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/dwiresp/csf_response.txt', '0d44fe75277631b95d293817b294b299')), ('gm_odf', 'gm.mif'), ('gm_txt', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/dwiresp/gm_response.txt', '0d44fe75277631b95d293817b294b299')), ('in_bval', ('/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A201/ses-shapesV1/dwi/sub-A201_ses-shapesV1_dwi.bval', '491229ee49ec51038771c213c2a66dee')), ('in_bvec', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A201/eddy_corrected.eddy_rotated_bvecs', 'cda3f9b8f166ec88665d3ad245010efe')), ('in_file', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A201/eddy_co

	 [Node] Hashes: [('algorithm', 'msmt_csd'), ('bval_scale', 'yes'), ('csf_odf', 'csf.mif'), ('csf_txt', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/csf_response.txt', '585090f0793a942643193d873d500f58')), ('gm_odf', 'gm.mif'), ('gm_txt', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/gm_response.txt', '2cc9afe925ee3aa371eb4641cd83357d')), ('in_bval', ('/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bval', '286321f2f3d3f560decafcf9e27e1531')), ('in_bvec', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.eddy_rotated_bvecs', '932715e667a99d2637f1975e0980d3f1')), ('in_file', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_co

dwi2fod -bvalue_scaling yes -fslgrad /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A201/eddy_corrected.eddy_rotated_bvecs /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A201/ses-shapesV1/dwi/sub-A201_ses-shapesV1_dwi.bval -lmax 8,8,8 msmt_csd /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A201/eddy_corrected.nii.gz /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/dwiresp/wm_response.txt wm.mif /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/dwiresp/gm_response.txt gm.mif /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/dwiresp/csf_response.txt csf.mif
190920-09:26:37,560 nipype.workflow INFO:
	 [Node] Running "mscsd" ("nipype.interfaces.mrtrix3.reconst.Esti

In [18]:
import os.path as op
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import dipy.data as dpd
from dipy.data import fetcher
import dipy.tracking.utils as dtu
import dipy.tracking.streamline as dts
from dipy.io.streamline import save_tractogram, load_tractogram
from dipy.stats.analysis import afq_profile, gaussian_weights
from dipy.io.stateful_tractogram import StatefulTractogram
from dipy.io.stateful_tractogram import Space

import AFQ.utils.streamlines as aus
import AFQ.data as afd
import AFQ.tractography as aft
import AFQ.registration as reg
import AFQ.dti as dti
import AFQ.segmentation as seg
from AFQ.utils.volume import patch_up_roi

ImportError: cannot import name 'afq_profile'

In [ ]:
# Create Nodes for AFQ Tractography Workflow
# Load processed data



In [ ]:
tg = load_tractogram('./dti_streamlines.trk', img)
streamlines = tg.streamlines

# streamlines = dts.Streamlines(
#     dtu.transform_tracking_output(streamlines,
#                                   np.linalg.inv(img.affine)))

templates = afd.read_templates()
bundle_names = ["UF", "CST"]

print("Segmenting fiber groups...")
segmentation = seg.Segmentation()
segmentation.segment(bundles,
                     streamlines,
                     fdata=hardi_fdata,
                     fbval=hardi_fbval,
                     fbvec=hardi_fbvec,
                     mapping=mapping,
                     reg_template=MNI_T2_img)


fiber_groups = segmentation.fiber_groups

print("Cleaning fiber groups...")
for bundle in bundles:
    fiber_groups[bundle] = seg.clean_fiber_group(fiber_groups[bundle])

for kk in fiber_groups:
    print(kk, len(fiber_groups[kk]))

    sft = StatefulTractogram(
        dtu.transform_tracking_output(fiber_groups[kk], img.affine),
        img, Space.RASMM)

    save_tractogram(sft, './%s_afq.trk'%kk,
                    bbox_valid_check=False)

print("Extracting tract profiles...")
for bundle in bundles:
    fig, ax = plt.subplots(1)
    weights = gaussian_weights(fiber_groups[bundle])
    profile = afq_profile(FA_data, fiber_groups[bundle],
                          np.eye(4), weights=weights)
    ax.plot(profile)
    ax.set_title(bundle)

plt.show()

In [ ]:
# (mscsd, datasink, [('wm_odf', '4_tract_Reconstruction.@par.@par.@par.@par'),
#                                        ('gm_odf', '4_tract_Reconstruction.@par.@par.@par.@par.@par'),
#                                        ('csf_odf','4_tract_Reconstruction.@par.@par.@par.@par.@par.@par')]),


# dwi_flow = Workflow(name = 'dwi_flow')
# dwi_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
#                   (sf, mrconv, [('dti', 'in_file')]),
#                   (mrconv, dwi_res, [('out_file', 'in_file')]),
#                   (sf, dwi_res, [('bval', 'in_bval'),
#                                  ('bvec', 'in_bvec')]),
#                   (dwi_res, datasink, [('csf_file', '4_DWI_Reconstruction.@par'),
#                                       ('wm_file', '4_DWI_Reconstruction.@par.@par'),
#                                       ('gm_file', '4_DWI_Reconstruction.@par.@par.@par')]),
#                   (sf, ms_csd, [('dti','in_file'),
#                                ('bval','in_bval'),
#                                ('bvec','in_bvec'),
#                                ('mask', 'mask_file')]),
#                   (dwi_res, ms_csd, [('wm_file', 'wm_txt'),
#                                      ('gm_file', 'gm_txt'),
#                                      ('csf_file', 'csf_txt')]),
#                   (ms_csd, datasink, [('wm_odf', '4_DWI_Reconstruction.@par.@par.@par.@par'),
#                                      ('gm_odf','4_DWI_Reconstruction.@par.@par.@par.@par.@par'),
#                                      ('csf_odf', '4_DWI_Reconstruction.@par.@par.@par.@par.@par.@par')]) 
#                  ])
# dwi_flow.base_dir = workflow_dir
# dwi_flow.write_graph(graph2use = 'flat')
# dwi = dwi_flow.run('MultiProc', plugin_args={'n_procs': 4})